In [115]:
%%time
import pandas as pd
from pathlib import Path
import sqlalchemy
import xlrd
import numpy as np
import pymysql.cursors

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 45.3 µs


# Specify your parameters

In [116]:
state_string = "NJ"
state_number_zillow = '34' #Make sure this is a string, with a leading 0 if 1 digit
asmt_or_trans = "asmt" #asmt or ztrans
zAsmt_or_Ztrans = "Z" + asmt_or_trans[0].upper() + asmt_or_trans[1:]
a_or_t_string = {"asmt": "Assessor", "trans": "Transaction"}[asmt_or_trans]
dataset = "Main"

# Create the Database

In [117]:
# Connect to a local SQLite database. SQLite is a simple file oriented database management "library." 
# in the following we launch a SQLite database stored in the .db file in the local folder.
db_name = state_string + "_" + asmt_or_trans + "_" + dataset
#from pathlib import Path
#dbfile = Path("/scratch/public/ethanburrell/"+db_name+".db")

sqlite_uri = "sqlite:///"+db_name+".db"
sqlite_engine = sqlalchemy.create_engine(sqlite_uri)

In [118]:
#Only Run this if you need to restart for whatever reason
#sqlite_engine.execute("DROP TABLE main;")
#sqlite_engine.execute("DROP TABLE PropertyInfo;")

# Create the header/columns for the main table

In [119]:
"""
This cell uses a library called xlrd to open and proccess .xlsx spreadsheets. 
I create a workbook from the layout data, and then select the "ZTrans" sheet that
is specific to the transaction data. This will get us the columns (and metadata) to 
create the table in sql.
"""
def create_header_information(path, sheet_name, table_name):
    workbook = xlrd.open_workbook(path)
    worksheet = workbook.sheet_by_name(sheet_name)
    header = [worksheet.cell(0, i).value for i in range(0,worksheet.ncols)]
    collumn_table = pd.DataFrame(columns=header)
    #display(collumn_table)
    for i in range(worksheet.nrows):
        if worksheet.cell(i, 0).value == table_name:
            info = [worksheet.cell(i, j).value for j in range(0,worksheet.ncols)]
            #print(info)
            collumn_table.loc[i] = info
            #df = pd.DataFrame(data=info, columns=collumn_list)
            #collumn_table.append(df)
            #collumn_list.append(worksheet.cell(i, 1).value)
    #collumn_table["FieldName"].to_array()
    return collumn_table

The "ut table name" is how we are identifying the table to find the collumn values. The "ZTrans" specifies the sheet in the Layout.xlxs file (either "ZAsmt", "ZTrans") 

In [120]:
%%time
layout_file_name = {"asmt": "LayoutAsmtHistory", "trans": "Layout"}[asmt_or_trans]
collumn_table = create_header_information('/scratch/public/ztrax/Zillow_' + str(a_or_t_string) +'/' + layout_file_name + '.xlsx', zAsmt_or_Ztrans, "ut" + dataset)
#collumn_table

CPU times: user 400 ms, sys: 4 ms, total: 404 ms
Wall time: 402 ms


Now we are going to use the "collumn_table" created above to create the collumns for a sqllite table.

In [121]:
#functions to help us create the table
def get_fields(path, table_name):
    layouts = pd.read_excel(path, sheet=1)
    layout = layouts[layouts['TableName'] == ('ut'+table_name)]
    field_names = layout.FieldName
    return field_names.values, layouts

def get_sql_fields(table_name, sql_name, layouts):
    layout = layouts[layouts['TableName'] == table_name]
    #print(layout)
    fields = layout['FieldName'] + ' ' + layout['DateType'] + '(' + (layout['MaxLength']).astype(str) + ')'
    if table_name == 'utBuilding':
        fields = fields.str.replace(r'RowID', 'RowID2')
        fields = fields.str.replace(r'BatchID', 'BatchID2')
        fields = fields.str.replace(r'FIPS', 'FIPS2')
        fields = fields.str.replace(r'LoadID', 'LoadID2')
    return ('CREATE TABLE ' + sql_name + '( ' + fields.str.cat(sep=', ') + ');')

In [122]:
# We use this function when we get errors like "Python Integer too large"
def get_sql_fields2(table_name, sql_name, layouts):
    layout = layouts[layouts['TableName'] == table_name]
    #print(layout)
    fields = layout['FieldName'] + ' ' + "TEXT"
    if table_name == 'utBuilding':
        fields = fields.str.replace(r'RowID', 'RowID2')
        fields = fields.str.replace(r'BatchID', 'BatchID2')
        fields = fields.str.replace(r'FIPS', 'FIPS2')
        fields = fields.str.replace(r'LoadID', 'LoadID2')
    return ('CREATE TABLE ' + sql_name + '( ' + fields.str.cat(sep=', ') + ');')

In [123]:
%%time
#This creates a string that we can use to create the info
main_sql_header = get_sql_fields('ut' + dataset, dataset, collumn_table)
print(main_sql_header)

CREATE TABLE Main( RowID uniqueidentifier(16.0), ImportParcelID bigint(8.0), FIPS char(5.0), State char(2.0), County varchar(50.0), ValueCertDate date(3.0), ExtractDate varchar(6.0), Edition smallint(2.0), ZVendorStndCode char(3.0), AssessorParcelNumber varchar(50.0), DupAPN char(2.0), UnformattedAssessorParcelNumber varchar(50.0), ParcelSequenceNumber int(4.0), AlternateParcelNumber varchar(50.0), OldParcelNumber varchar(50.0), ParcelNumberTypeStndCode char(1.0), RecordSourceStndCode char(1.0), RecordTypeStndCode char(2.0), ConfidentialRecordFlag char(1.0), PropertyAddressSourceStndCode char(1.0), PropertyHouseNumber varchar(13.0), PropertyHouseNumberExt varchar(10.0), PropertyStreetPreDirectional char(2.0), PropertyStreetName varchar(50.0), PropertyStreetSuffix varchar(6.0), PropertyStreetPostDirectional char(2.0), PropertyFullStreetAddress varchar(80.0), PropertyCity varchar(45.0), PropertyState char(2.0), PropertyZip varchar(5.0), PropertyZip4 char(4.0), OriginalPropertyFullStreetA

Replace all BigInts with Strings, because there is data that cannot fit in the BigInt type

In [124]:
main_sql_header

'CREATE TABLE Main( RowID uniqueidentifier(16.0), ImportParcelID bigint(8.0), FIPS char(5.0), State char(2.0), County varchar(50.0), ValueCertDate date(3.0), ExtractDate varchar(6.0), Edition smallint(2.0), ZVendorStndCode char(3.0), AssessorParcelNumber varchar(50.0), DupAPN char(2.0), UnformattedAssessorParcelNumber varchar(50.0), ParcelSequenceNumber int(4.0), AlternateParcelNumber varchar(50.0), OldParcelNumber varchar(50.0), ParcelNumberTypeStndCode char(1.0), RecordSourceStndCode char(1.0), RecordTypeStndCode char(2.0), ConfidentialRecordFlag char(1.0), PropertyAddressSourceStndCode char(1.0), PropertyHouseNumber varchar(13.0), PropertyHouseNumberExt varchar(10.0), PropertyStreetPreDirectional char(2.0), PropertyStreetName varchar(50.0), PropertyStreetSuffix varchar(6.0), PropertyStreetPostDirectional char(2.0), PropertyFullStreetAddress varchar(80.0), PropertyCity varchar(45.0), PropertyState char(2.0), PropertyZip varchar(5.0), PropertyZip4 char(4.0), OriginalPropertyFullStreet

In [125]:
%%time
sqlite_engine.execute(main_sql_header)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 10.1 ms


Check that the collumn is in the database

In [126]:
%%time
pd.read_sql("SELECT * FROM " + dataset, sqlite_engine)

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 18.9 ms


,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,...,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,SourceChkSum


# Inserting the Main.txt data into the database

In [127]:
#path = "/accounts/projects/mzuk/ethanburrell/Documents/Research/main_sample.txt"
fips_string = ["34003", "34013", "34017", "34019", "34021", "34023", "34025", "34027", "34029", "34031", "34035", "34037", "34039", "34041", "36005", "36027", "36047", "36059", "36061", "36071", "36079", "36081", "36085", "36087", "36103", "36105", "36111", "36119", "09001", "09005", "09009"] #"9001", "9005", "9009"
fips_int = [int(i) for i in fips_string]

Please note that the following cell will take a while to run ~~(1h 18min 9s for NJ and 45min 22s for CT).~~

I think I greatly improved performace

# YOU MUST RUN THIS TO GET THE BEST RESULT

To get the length varible in the next cell you need to open terminal, log onto quesnay and run
```
>>> cd /scratch/public/ztrax/Zillow_Transaction/<state number here>/<ZTrans or ZAsmt>/
>>> wc -l Main.txt
9998468 Main.txt
```
This number is the number of rows in the CSV file, use this as the length argument.

In [128]:
%%time
#6128765
# CT Trans Main = 5348354, CT Trans Propertyinfo = 5361730, CT Asmt Main  = 1335587
#ny asmt main = 63491606
# NJ Asmt main = 32293071
length = 32293071 #This is the number that you got from above
# Works on partitions of the dataset
def work(path, fields, partitions, rows, fn):
    div = rows // partitions

    parts = np.arange(0, rows, div)
    for i in parts:
        df = pd.read_csv(path, sep='|', names=fields, skiprows=i, nrows=div, encoding='latin-1', dtype=str)
        fn(df)
        del df
        
def to_sql(df):
    # We can use the pandas.to_sql command to put the data in our SQLlite database:
    print(len(df.loc[df["FIPS"].isin(fips_string)]))
    #print(type(df["FIPS"][0]))
    df.loc[df["FIPS"].isin(fips_string)].to_sql(dataset, sqlite_engine, if_exists='append', index=False)
    #df.to_sql(dataset, sqlite_engine, if_exists='append', index=False)

work("/scratch/public/ztrax/Zillow_" + a_or_t_string + "/" + state_number_zillow + "/"+ zAsmt_or_Ztrans +"/" + dataset + '.txt', collumn_table["FieldName"].values, 5000, length, to_sql)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4497
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6458
6

OperationalError: (sqlite3.OperationalError) unable to open database file [SQL: 'INSERT INTO "Main" ("RowID", "ImportParcelID", "FIPS", "State", "County", "ValueCertDate", "ExtractDate", "Edition", "ZVendorStndCode", "AssessorParcelNumber", "DupAPN", "UnformattedAssessorParcelNumber", "ParcelSequenceNumber", "AlternateParcelNumber", "OldParcelNumber", "ParcelNumberTypeStndCode", "RecordSourceStndCode", "RecordTypeStndCode", "ConfidentialRecordFlag", "PropertyAddressSourceStndCode", "PropertyHouseNumber", "PropertyHouseNumberExt", "PropertyStreetPreDirectional", "PropertyStreetName", "PropertyStreetSuffix", "PropertyStreetPostDirectional", "PropertyFullStreetAddress", "PropertyCity", "PropertyState", "PropertyZip", "PropertyZip4", "OriginalPropertyFullStreetAddress", "OriginalPropertyAddressLastline", "PropertyBuildingNumber", "PropertyZoningDescription", "PropertyZoningSourceCode", "CensusTract", "TaxIDNumber", "TaxAmount", "TaxYear", "TaxDelinquencyFlag", "TaxDelinquencyAmount", "TaxD

Check to see if all of the data is there

In [77]:
pd.set_option('display.max_columns', None)  
pd.read_sql("SELECT * FROM "+ dataset +" LIMIT 10", sqlite_engine)

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,UnformattedAssessorParcelNumber,ParcelSequenceNumber,AlternateParcelNumber,OldParcelNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,PropertyZip4,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,SourceChkSum


After this, run (in the root of quesnay)
```
mv /accounts/projects/mzuk/ethanburrell/Documents/Research/<STATE>_trans.db /scratch/public/ethanburrell
```
This moves the file into the scratch folder this way I don't hit my user disk quota

# Rinse and Repeat for all the types of Data You have to collect

In [81]:
sqlite_engine.execute("DROP TABLE Main;")
